In [1]:
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

C:\Users\kevin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
text_length = 40
vocab_size = None

In [3]:
#input_shape = (text_length, vocab_size)

In [4]:
from keras.preprocessing.text import Tokenizer

In [5]:
t = Tokenizer(filters='\n', num_words=vocab_size) #try setting vocab_size to 2000 or 4000 if training not working
with open("len40.txt") as f:
    t.fit_on_texts([f.read(), '<m>'])

In [6]:
X = []
with open("len40_mask3.txt") as f:
    for line in f:
        X.append(t.texts_to_sequences([line])[0])

In [7]:
X = np.array(X) #turn our data from python list of lists into np array

In [8]:
X_filled = []
with open("len40.txt") as f:
    for line in f:
        X_filled.append(t.texts_to_sequences([line])[0])


In [9]:
X_filled = np.array(X_filled)

In [10]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder()
OHE.fit(np.array(list(t.word_index.values())).reshape((-1,1)))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [11]:
#OHE.transform(X_filled[0].reshape((-1,1))).todense()

In [12]:
reverse_word_map = dict(map(reversed, t.word_index.items())) #turn our numbers into words

In [13]:
#[reverse_word_map[i] for i in X[0]] #test

In [14]:
from keras.layers import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed

In [15]:
if vocab_size is None:
    vocab_size = len(t.word_index)

In [16]:
from keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from keras.models import Model

In [17]:
main_input_g = Input(shape=(text_length,vocab_size))

In [18]:
#embedding_layer = Embedding(vocab_size, 300, input_length=text_length)
embedding_layer = TimeDistributed(Dense(300))

In [19]:
x = embedding_layer(main_input_g)

In [20]:
x = Bidirectional(LSTM(300, return_sequences=True))(x)

In [21]:
main_output_g = TimeDistributed(Dense(vocab_size, activation='softmax'))(x)

In [22]:
generator_model = Model(inputs=[main_input_g], outputs=[main_output_g])

In [23]:
generator_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 97491)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 300)           29247600  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 600)           1442400   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 97491)         58592091  
Total params: 89,282,091
Trainable params: 89,282,091
Non-trainable params: 0
_________________________________________________________________


In [24]:
main_input_d = Input(shape=(text_length,vocab_size))
aux_input_d = Input(shape=(text_length,vocab_size))

In [25]:
filled_in = embedding_layer(main_input_d)
context = embedding_layer(aux_input_d)

In [26]:
combined = Concatenate(axis=-1)([filled_in, context])

In [27]:
x = Bidirectional(LSTM(300, return_sequences=True))(combined)

In [28]:
main_output_d = TimeDistributed(Dense(1, activation='sigmoid'))(x) #or sigmoid

In [29]:
discriminator_model = Model(inputs = [main_input_d, aux_input_d], outputs=[main_output_d])

In [30]:
discriminator_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40, 97491)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 40, 97491)    0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 40, 300)      29247600    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 40, 600)      0           time_distributed_1[1][0]         
          

In [31]:
#AM = Sequential()
optimizer = RMSprop(lr=0.0001, decay=3e-8)
AM_input = Input(shape=(text_length,vocab_size))
generator_output = generator_model(AM_input)
discriminator_output = discriminator_model([generator_output, AM_input])
AM = Model(inputs=[AM_input], outputs=discriminator_output)
#AM.add(discriminator_model)
AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
    metrics=['accuracy'])
AM.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 40, 97491)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 40, 97491)    89282091    input_4[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 40, 1)        31410601    model_1[1][0]                    
                                                                 input_4[0][0]                    
Total params: 91,445,092
Trainable params: 91,445,092
Non-trainable params: 0
__________________________________________________________________________________________________


In [33]:
# from keras.callbacks import ModelCheckpoint
# checkpointAM = ModelCheckpoint("AM", verbose=1, save_best_only=False,period=5)
# checkpointDM = ModelCheckpoint("DM", verbose=1, save_best_only=False,period=5)

In [40]:
import gc
def train(train_steps=2000, batch_size=10, print_interval=2, save_interval=50):
    def categorical_to_one_hot(categorical):
        return OHE.transform(categorical.reshape(batch_size*text_length,1)).toarray().reshape(batch_size,text_length,-1)
    noise_input = None
    if save_interval>0:
        noise_input = np.random.normal(0, 1, size=[16, 100])
    for i in range(train_steps):
        if i%2 == 0:
            gc.collect()
        for j in range(10):
            indices_fake = np.random.randint(0, X.shape[0], size=batch_size)
            fake_unfilled = X[indices_fake, :]
            #noise = np.random.normal(0, 1, size=[batch_size, 100])
            fake_unfilled = categorical_to_one_hot(fake_unfilled)
            fake_filled = generator_model.predict(fake_unfilled)

            indices_real = np.random.randint(0, X.shape[0], size=batch_size)
            real_unfilled = X[indices_real, :]
            real_filled = X_filled[indices_real, :]
            real_unfilled = categorical_to_one_hot(real_unfilled)
            real_filled = categorical_to_one_hot(real_filled)

            x_unfilled = np.concatenate((real_unfilled, fake_unfilled))
            x_filled = np.concatenate((real_filled, fake_filled))
            y = np.ones([2*batch_size, text_length, 1])
            y[batch_size:, :, :] = 0
            d_loss = discriminator_model.train_on_batch([x_filled, x_unfilled], y)
            if d_loss[0] < 0.71:
                break
        #above here is training the discriminator (DM)
        #under here is training the generator (AM)
        for j in range(10):
            y = np.ones([batch_size, text_length, 1])
            #noise = np.random.normal(0, 1, size=[batch_size, 100])
            indices_fake = np.random.randint(0, X.shape[0], size=batch_size)
            fake_unfilled = X[indices_fake, :]
            fake_unfilled = categorical_to_one_hot(fake_unfilled)
            a_loss = AM.train_on_batch(fake_unfilled, y)
            if a_loss[0] < 1.2:
                break
        log_mesg = "*%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
        print(log_mesg)
        if print_interval>0:
            if (i+1)%print_interval==0:
                print_examples(indices_fake, fake_filled, fake_unfilled, save2file=True)#samples=noise_input.shape[0],\
        if save_interval>0:
            if (i+1)%save_interval == 0:
                AM.save("AM_save.hdf5")
                discriminator_model.save("DM_save.hdf5")
                    #noise=noise_input, step=(i+1))

In [41]:
def print_examples(indices_fake, fake_filled, fake_unfilled, save2file = False, ):
    batch_size = 10
    def categorical_to_one_hot(categorical):
        return OHE.transform(categorical.reshape(batch_size*text_length,1)).toarray().reshape(batch_size,text_length,-1)

    print(' '.join([reverse_word_map.get(OHE.active_features_[np.argmax(i)], '<m>') for i in fake_filled[0]]))
    print(' '.join([reverse_word_map.get(i) for i in X_filled[indices_fake][0]]))
    print(' '.join([reverse_word_map.get(i) for i in X[indices_fake][0]]))

In [ ]:
optimizer = RMSprop(lr=0.0002, decay=6e-8)

discriminator_model.compile(loss='binary_crossentropy', optimizer=optimizer,\
    metrics=['accuracy'])

train(train_steps=10000)

*0: [D loss: 0.695378, acc: 0.476250]  [A loss: 0.737415, acc: 0.057500]
*1: [D loss: 0.683271, acc: 0.573750]  [A loss: 0.728400, acc: 0.117500]
near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near dressed dressed dressed dressed
na ride eos yeah i-i wan na ride eos ( by the ocean side ) eos i wan na ride eos i-i wan na ride with you eos i-i wan na ride eos i wan na ride eos i just
na ride eos yeah i-i wan na ride eos ( by <m> <m> side ) eos i wan na ride eos i-i wan na ride with you eos i-i wan na ride <m> i wan na ride eos i just
*2: [D loss: 0.680848, acc: 0.625000]  [A loss: 0.737214, acc: 0.037500]
*3: [D loss: 0.677401, acc: 0.668750]  [A loss: 0.746040, acc: 0.010000]
near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near near dre

In [94]:
#one_hot_encoded_test = OHE.transform(fake_unfilled.reshape(400,1)).toarray().reshape(10,40,-1)

#[reverse_word_map.get(OHE.active_features_[np.argmax(i)], '<m>') for i in one_hot_encoded_test[0]]

In [104]:
# batch_size = 10
# def categorical_to_one_hot(categorical):
#     return OHE.transform(categorical.reshape(batch_size*text_length,1)).toarray().reshape(batch_size,text_length,-1)

# indices_fake = np.random.randint(0, X.shape[0], size=10)
# fake_unfilled = X[indices_fake, :]
# #noise = np.random.normal(0, 1, size=[batch_size, 100])
# fake_unfilled = categorical_to_one_hot(fake_unfilled)
# fake_filled = generator_model.predict(fake_unfilled)
# print(' '.join([reverse_word_map.get(OHE.active_features_[np.argmax(i)], '<m>') for i in fake_filled[0]]))
# print(' '.join([reverse_word_map.get(i) for i in X_filled[indices_fake][0]]))
# print(' '.join([reverse_word_map.get(i) for i in X[indices_fake][0]]))

ice sguardo honey honey honey honey hurricane pie pie honey pie pie honey honey honey honey pie pie honey pie pie honey honey honey honey honey honey honey honey honey pie pie pie pie ice xzibit honey honey darlin honey
do n't need a crystal ball for me to see clearly eos no astrology or tarot cards eos watching cnn and holding my breath eos to face the daily news scares me to death eos i 'm pessi-mystic eos i
do n't need a crystal ball <m> me to see clearly eos no <m> or tarot cards eos watching cnn and holding my breath eos to face the daily news scares <m> to death eos i 'm pessi-mystic eos i
